In [3]:
import tensorflow as tf
import numpy as np
AUTO = tf.data.experimental.AUTOTUNE
GCS_OUTPUT = 'gs://cloudfire_lyrical-edition-273206/fire_dataset/tfrecords-dataset-8/'  # prefix for output file names

TARGET_SIZE = [256, 256]


def read_tfrecord(example):
    features = {
        "image": tf.io.FixedLenFeature([], tf.string),  # tf.string = bytestring (not text string)
        "class": tf.io.FixedLenFeature([], tf.int64),  # shape [] means scalar

        # additional (not very useful) fields to demonstrate TFRecord writing/reading of different types of data
        "label": tf.io.FixedLenFeature([], tf.string),  # one bytestring
        "size": tf.io.FixedLenFeature([2], tf.int64),  # two integers
        "one_hot_class": tf.io.VarLenFeature(tf.float32)  # a certain number of floats
    }
    # decode the TFRecord
    example = tf.io.parse_single_example(example, features)

    # FixedLenFeature fields are now ready to use: exmple['size']
    # VarLenFeature fields require additional sparse_to_dense decoding

    image = tf.image.decode_jpeg(example['image'], channels=3)
    image = tf.reshape(image, [*TARGET_SIZE, 3])

    class_num = example['class']

    label = example['label']
    height = example['size'][0]
    width = example['size'][1]
    one_hot_class = tf.sparse.to_dense(example['one_hot_class'])
    return image, class_num, label, height, width, one_hot_class


# read from TFRecords. For optimal performance, read from multiple
# TFRecord files at once and set the option experimental_deterministic = False
# to allow order-altering optimizations.

option_no_order = tf.data.Options()
option_no_order.experimental_deterministic = False

filenames = tf.io.gfile.glob(GCS_OUTPUT + "*.tfrec")
dataset4 = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO)
dataset4 = dataset4.with_options(option_no_order)
dataset4 = dataset4.map(read_tfrecord, num_parallel_calls=AUTO)

In [4]:
dataset4

<ParallelMapDataset shapes: ((256, 256, 3), (), (), (), (), (None,)), types: (tf.uint8, tf.int64, tf.string, tf.int64, tf.int64, tf.float32)>

In [6]:
for image, class_num, label, height, width, one_hot_class in dataset4:
    print("Image shape {}, {}x{} px, class={} ({:>10}, {})".format(image.numpy().shape, width, height, class_num,label.numpy().decode('utf8'),one_hot_class))

Image shape (256, 256, 3), 256x256 px, class=1 (    Normal, [0. 1.])
Image shape (256, 256, 3), 256x256 px, class=0 (      Fire, [1. 0.])
Image shape (256, 256, 3), 256x256 px, class=0 (      Fire, [1. 0.])
Image shape (256, 256, 3), 256x256 px, class=1 (    Normal, [0. 1.])
Image shape (256, 256, 3), 256x256 px, class=0 (      Fire, [1. 0.])
Image shape (256, 256, 3), 256x256 px, class=1 (    Normal, [0. 1.])
Image shape (256, 256, 3), 256x256 px, class=1 (    Normal, [0. 1.])
Image shape (256, 256, 3), 256x256 px, class=1 (    Normal, [0. 1.])
Image shape (256, 256, 3), 256x256 px, class=1 (    Normal, [0. 1.])
Image shape (256, 256, 3), 256x256 px, class=0 (      Fire, [1. 0.])
Image shape (256, 256, 3), 256x256 px, class=1 (    Normal, [0. 1.])
Image shape (256, 256, 3), 256x256 px, class=1 (    Normal, [0. 1.])
Image shape (256, 256, 3), 256x256 px, class=0 (      Fire, [1. 0.])
Image shape (256, 256, 3), 256x256 px, class=0 (      Fire, [1. 0.])
Image shape (256, 256, 3), 256x256

In [7]:
def dataset_to_numpy_util(dataset, N):
    dataset = dataset.unbatch().batch(N)
    for images, labels in dataset:
        numpy_images = images.numpy()
        numpy_labels = labels.numpy()
        break;  
    return numpy_images, numpy_labels
'''
def title_from_label_and_target(label, correct_label):
    label = np.argmax(label, axis=-1)  # one-hot to class number
    correct_label = np.argmax(correct_label, axis=-1) # one-hot to class number
    correct = (label == correct_label)
    return "{} [{}{}{}]".format(CLASSES[label], str(correct), ', shoud be ' if not correct else '',
                                CLASSES[correct_label] if not correct else ''), correct
'''
def display_one_flower(image, title, subplot, red=False):
    plt.subplot(subplot)
    plt.axis('off')
    plt.imshow(image)
    plt.title(title, fontsize=16, color='red' if red else 'black')
    return subplot+1
  
def display_9_images_from_dataset(dataset):
    subplot=331
    plt.figure(figsize=(13,13))
    images, labels = dataset_to_numpy_util(dataset, 9)
    for i, image in enumerate(images):
        title = CLASSES[np.argmax(labels[i], axis=-1)]
        subplot = display_one_flower(image, title, subplot)
        if i >= 8:
            break;
              
    plt.tight_layout()
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.show()  

    '''
def display_9_images_with_predictions(images, predictions, labels):
    subplot=331
    plt.figure(figsize=(13,13))
    for i, image in enumerate(images):
        title, correct = title_from_label_and_target(predictions[i], labels[i])
        subplot = display_one_flower(image, title, subplot, not correct)
        if i >= 8:
            break;
              
    plt.tight_layout()
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.show()
    
def display_training_curves(training, validation, title, subplot):
    if subplot%10==1: # set up the subplots on the first call
        plt.subplots(figsize=(10,10), facecolor='#F0F0F0')
        plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.set_facecolor('#F8F8F8')
    ax.plot(training)
    ax.plot(validation)
    ax.set_title('model '+ title)
    ax.set_ylabel(title)
    #ax.set_ylim(0.28,1.05)
    ax.set_xlabel('epoch')
    ax.legend(['train', 'valid.'])
    '''

display_9_images_from_dataset(dataset4)    
    

ValueError: in converted code:

    /opt/conda/lib/python3.7/site-packages/tensorflow_core/python/data/ops/dataset_ops.py:1966 normalize
        (arg,) + rest)
    /opt/conda/lib/python3.7/site-packages/tensorflow_core/python/data/util/structure.py:350 to_batched_tensor_list
        component), element_spec, element)
    /opt/conda/lib/python3.7/site-packages/tensorflow_core/python/data/util/structure.py:324 _to_tensor_list_helper
        reduce_fn, zip(nest.flatten(element_spec), nest.flatten(element)), [])
    /opt/conda/lib/python3.7/site-packages/tensorflow_core/python/data/util/structure.py:321 reduce_fn
        return encode_fn(state, spec, component)
    /opt/conda/lib/python3.7/site-packages/tensorflow_core/python/data/util/structure.py:350 <lambda>
        component), element_spec, element)
    /opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/type_spec.py:462 _to_batched_tensor_list
        raise ValueError("Value %s has insufficient rank for batching." 

<Figure size 936x936 with 0 Axes>